# Import

In [32]:
import numpy as np
import random
from scipy.stats import spearmanr, pearsonr
from scipy.optimize import fsolve
import math

#import the custom functions
import MFunctions as mf

In [2]:
#Load verb_symilarity which is the same for all
simdict = mf.verb_similarity("SimVerb-3500.txt")

# Define theoretical expectation values

In [62]:
# Since these expectation values depend on both, the considered set of matrices as well as the set of observables
#we tak an ad hoc approach for the time being

#The expectation values of the higher order observables are computed starting from those of the linear and quadratic observables
#Firts extract this in the case of each single set of matrices we are interested in

# Make observables object
set1 = mf.observables(["Linear.txt","Quadratic.txt"])
# Load matrices and make vector dictionaries
objsubset1 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set1)
subobjset1 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set1,[0])
obj08sub02set1 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set1,[1])
obj09sub01set1 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set1,[1])
obj05sub05set1 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.5,0.5]],set1,[1])
obj02sub08set1 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set1,[1])

In [63]:
#The following system provides the parameters. y is the parameters to solve for, D is the dimension of the matrices, mat_exp is the
#mean of the linear and quadratic observables vector on the given set of matrices
#For us D=100 if I ma not mistaking

def system_to_solve(y,D,mat_exp):
    f1 = y[0] + (np.sqrt(D-1))*y[1] - float(mat_exp[0])  # <sum_{i} M_{ii}>
    f2 = D*y[0] - float(mat_exp[1])  # <sum_{i,j} M_{ij}>
    f3 = y[0]**2 + y[1]**2 + y[2] + y[4] + (D-1)*y[8] + (D-1)*y[10] + (D-1)*y[5] + ((D*(D-3))/2)*y[11] +\
        (((D-1)*(D-2))/2)*y[12] - float(mat_exp[2])  # <sum_{i,j} M_ij M_ij>
    f4 = ((D*(D-3))/2)*y[11] - (((D-1)*(D-2))/2)*y[12] + 2*(D-1)*y[6] + (D-1)*y[10] + y[2] + y[4] + y[0]**2 +\
        y[1]**2 - float(mat_exp[3])  # <sum_{i,j} M_ij M_ji>
    f5 = y[2] + np.sqrt(D-1)*y[3] + (D-1)*y[6] + (D-1)*y[8] + (D-1)*(np.sqrt(D-2))*y[9] + y[0]**2 +\
        y[0]*y[1]*(np.sqrt(D-1)) - float(mat_exp[4])  # <sum_{i,j} M_ii M_ij>
    f6 = y[2] + np.sqrt(D-1)*y[3] + (D-1)*y[6] + (D-1)*y[5] + (D-1)*(np.sqrt(D-2))*y[7] + y[0]**2 +\
        y[0]*y[1]*(np.sqrt(D-1)) - float(mat_exp[5])  # <sum_{i,j} M_ii M_ji>
    f7 = D*y[2] + D*(D-1)*y[8] + D*(y[0]**2) - float(mat_exp[6])  # <sum_{i,j,k} M_ij M_ik>
    f8 = D*y[2] + D*(D-1)*y[5] + D*(y[0]**2) - float(mat_exp[7])  # <sum_{i,j,k} M_ij M_kj>
    f9 = D*y[2] + D*(D-1)*y[6] + D*(y[0]**2) - float(mat_exp[8])  # <sum_{i,j,k} M_ij M_jk>
    f10 = (D**2)*y[2] + (D**2)*(y[0]**2) - float(mat_exp[9])  # <sum_{i,j,k,l} M_ij M_kl>
    f11 = (D**-1)*y[2] + ((D-1)/D)*y[4] + 2*((np.sqrt(D-1))/D)*y[3] + ((D-1)/D)*y[5] + ((D-1)/D)*y[8] +\
        ((D-1)/D)*(D-2)*y[10] + 2*((D-1)/D)*y[6] + 2*((D-1)/D)*(np.sqrt(D-2))*y[7] +\
        2*((D-1)/D)*(np.sqrt(D-2))*y[9] + ((y[0]**2)/D) + 2*((np.sqrt(D-1))/D)*y[0]*y[1] +\
        ((D-1)/D)*(y[1]**2) - float(mat_exp[10])  # <sum_{i} M_ii M_ii OR (M_ii)^2>
    f12 = y[2] + (D-1)*y[4] + 2*(np.sqrt(D-1))*y[3] + y[0]**2 + 2*(np.sqrt(D-1))*y[0]*y[1] +\
        (D-1)*(y[1]**2) - float(mat_exp[11])  # <sum_{i,j} M_ii M_jj>
    f13 = D*y[2] + D*(np.sqrt(D-1))*y[3] + D*(y[0]**2) +\
        D*(np.sqrt(D-1))*y[0]*y[1] - float(mat_exp[12])  # <sum_{i,j,k} M_{ii} M_{jk}>

    return [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13]

In [ ]:
#If we call the solution to the above system x then x is an array containing the solved model parameters.
# Each parameter as defined in PIGMM is associated to each x[i] as follows:
# x[0] = tilde(mu)_1
# x[1] = tilde(mu)_2
# x[2] = (LamV0^-1)_11
# x[3] = (LamV0^-1)_12
# x[4] = (LamV0^-1)_22
# x[5] = (LamVH^-1)_11
# x[6] = (LamVH^-1)_12
# x[7] = (LamVH^-1)_13
# x[8] = (LamVH^-1)_22
# x[9] = (LamVH^-1)_23
# x[10] = (LamVH^-1)_33
# x[11] = (LamV2^-1)
# x[12] = (LamV3^-1)

In [64]:
# Theoretical expectation values - the following graph labelling system is used to match that of the GTMDS paper.

def theory_expectations(x,D):

   # G1: Exp Val for sum_{i} M_{ii}^3
    G1 = 3*((x[0]/D) + ((np.sqrt(D-1))/D)*x[1]) * ((x[2]/D) + ((D-1)/D)*x[4] + 2*(np.sqrt(D-1)/D)*x[3] + ((D-1)/D)*x[5] +\
        ((D-1)/D)*x[8] + ((D-1)/D)*(D-2)*x[10] + 2*((D-1)/D)*x[6] + 2*((D-1)/D)*(np.sqrt(D-2))*x[7] +\
        2*((D-1)/D)*(np.sqrt(D-2))*x[9]) + (D**-2)*((x[0] + (np.sqrt(D-1))*x[1])**3)

    # G2: Exp Val for sum_{i,j} M_{ij}^3
    G2 = (x[0]**3)/D + (3/D)*x[0]*(x[1]**2) + ((D-2)/(D*(np.sqrt(D-1))))*(x[1]**3) + 3*(x[0]/D)*(x[2] + x[4] +\
        (D-1)*x[8] + (D-1)*x[10] + (D-1)*x[5] + ((D*(D-3))/2)*x[11] + (((D-1)*(D-2))/2)*x[12]) +\
        3*x[1]*x[4]*((D-2)/(D*(np.sqrt(D-1)))) + (6*x[1]*x[3])/D + 3*x[1]*x[10]*((D-3)/D)*(np.sqrt(D-1)) +\
        6*x[1]*x[6]*((np.sqrt(D-1))/D) + 6*x[1]*x[7]*((np.sqrt((D-1)*(D-2)))/D) +\
        6*x[1]*x[9]*((np.sqrt((D-1)*(D-2)))/D) + 3*x[1]*x[11]*((-D**2 + 3*D)/(2*D*np.sqrt(D-1))) -\
        (3/2)*x[1]*x[12]*(((D-2)*np.sqrt(D-1))/D)

    # G3: Exp Val for sum_{i,j,k} M_{ij} M_{jk} M_{ki}
    G3 = x[0]**3 + (x[1]**3)/(np.sqrt((D-1))) + 3*x[0]*(x[2] + (D-1)*x[6]) + 3*(x[1]/(np.sqrt(D-1)))*x[4] +\
        3*x[1]*(np.sqrt(D-1))*x[10] + 3*(x[1])*(np.sqrt(D-1))*x[6] + 3*x[1]*x[11]*((D*(D-3))/(2*(np.sqrt(D-1)))) -\
        3*x[1]*x[12]*(((D-2)*(np.sqrt(D-1)))/2)

    # G4: Exp Val for sum_{i,j,k} M_{ij} M_{jj} M_{jk}
    G4 = x[0]*x[2] + np.sqrt(D-1)*x[0]*x[3] +(D-1)*x[0]*x[5] + (D-1)*x[0]*x[6] +(D-1)*np.sqrt(D-2)*x[0]*x[7] + x[0]*x[2] +\
         (D-1)*x[0]*x[6] + np.sqrt(D-1)*x[1]*x[2] + ((D-1)**(3/2))*x[1]*x[6] + x[0]*x[2] + np.sqrt(D-1)*x[0]*x[3] +\
         (D-1)*x[0]*x[6] + x[0]*(D-1)*x[8] + (D-1)*np.sqrt(D-2)*x[0]*x[9] + (x[0]**3) + (x[0]**2)*x[1]*np.sqrt(D-1)

    # G5: Exp value for sum_{i,j,k,l} M_{ij}M_{kk}M_{ll}
    G5 = 2*(D*x[2] + D*(np.sqrt(D-1))*x[3])*(x[0] + np.sqrt(D-1)*x[1]) + (x[2] + (D-1)*x[4] +\
         2*np.sqrt(D-1)*x[3] + ((x[0] + np.sqrt(D-1)*x[1])**2)) * D*x[0]

    # G6: Exp value for sum_{i,j,k,l} M_{ij}M_{jk}M_{ll}
    G6 = 3*D*x[0]*x[2] + D*np.sqrt(D-1)*x[2]*x[1] + D*(D-1)*x[6]*x[0] + D*((D-1)**(3/2))*x[6]*x[1] +\
         2*D*(np.sqrt(D-1))*x[0]*x[3] + D*(x[0]**3) + D*np.sqrt(D-1)*(x[0]**2)*x[1]

    # G7: Exp val for sum_{i,j,k,l,m} M_{ij}M_{kl}M_{mm}
    G7 = ((D**2)*x[2])*(x[0] + np.sqrt(D-1)*x[1]) + 2*(D*x[2] + D*np.sqrt(D-1)*x[3])*D*x[0] + ((D*x[0])**2)*(x[0] +\
         np.sqrt(D-1)*x[1])

    # G8: Exp Val for sum_{i,j,k,l,m,n} M_{ij} M{kl} M{mn}
    G8 = 3*x[0]*(D**3)*x[2] + (x[0]**3)*(D**3)

    # G9: Exp Val for sum_{i_(1,2,3,4,5,6,7)} M_{i_(1,2)} M_{i_(3,4)} M_{i_(5,6)}M_{i_(7,7)}
    G9 = 3*((D**2)*x[2]*(D*x[2] + D*np.sqrt(D-1)*x[3])) + 3*((D**2)*x[2]*(D*x[0])*(x[0] + np.sqrt(D-1)*x[1]) +\
        (D*x[2] + D*np.sqrt(D-1)*(x[3]))*(D*x[0])**2) + ((D*x[0])**3)*(x[0] + np.sqrt(D-1)*x[1])

    # G10: Exp Val for sum_{i_(1,2,3,4,5,6,7,8)} M_{i_(1,2)} M_{i_(3,4)} M_{i_(5,6)}M_{i_(7,8)}
    G10 = 3*(D**4)*(x[2]**2) + 6*(D**4)*x[2]*(x[0]**2) + (D*x[0])**4
    
    return(np.array([G1,G2,G3,G4,G5,G6,G7,G8,G9,G10]))

In [65]:
#Solve the system for each set of matrices
x1 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,objsubset1.mean))
x2 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,obj09sub01set1.mean))
x3 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,obj08sub02set1.mean))
x4 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,subobjset1.mean))
x5 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,obj05sub05set1.mean))
x6 = fsolve(system_to_solve,[0,0,0,0,0,0,0,0,0,0,0,0,0],(100,obj02sub08set1.mean))

In [66]:
thmean1 = theory_expectations(x1,100)
thmean2 = theory_expectations(x2,100)
thmean3 = theory_expectations(x3,100)
thmean4 = theory_expectations(x4,100)
thmean4a = theory_expectations(x5,100)
thmean4b = theory_expectations(x6,100)

## Add the additional 5 theoretical expectation values

In [67]:
# Theoretical expectation values - the following graph labelling system is used to match that of the GTMDS paper.

def theory_expectations2(x,D):

   # G1: Exp Val for sum_{i} M_{ii}^3
    G1 = 3*((x[0]/D) + ((np.sqrt(D-1))/D)*x[1]) * ((x[2]/D) + ((D-1)/D)*x[4] + 2*(np.sqrt(D-1)/D)*x[3] + ((D-1)/D)*x[5] +\
        ((D-1)/D)*x[8] + ((D-1)/D)*(D-2)*x[10] + 2*((D-1)/D)*x[6] + 2*((D-1)/D)*(np.sqrt(D-2))*x[7] +\
        2*((D-1)/D)*(np.sqrt(D-2))*x[9]) + (D**-2)*((x[0] + (np.sqrt(D-1))*x[1])**3)

    # G2: Exp Val for sum_{i,j} M_{ij}^3
    G2 = (x[0]**3)/D + (3/D)*x[0]*(x[1]**2) + ((D-2)/(D*(np.sqrt(D-1))))*(x[1]**3) + 3*(x[0]/D)*(x[2] + x[4] +\
        (D-1)*x[8] + (D-1)*x[10] + (D-1)*x[5] + ((D*(D-3))/2)*x[11] + (((D-1)*(D-2))/2)*x[12]) +\
        3*x[1]*x[4]*((D-2)/(D*(np.sqrt(D-1)))) + (6*x[1]*x[3])/D + 3*x[1]*x[10]*((D-3)/D)*(np.sqrt(D-1)) +\
        6*x[1]*x[6]*((np.sqrt(D-1))/D) + 6*x[1]*x[7]*((np.sqrt((D-1)*(D-2)))/D) +\
        6*x[1]*x[9]*((np.sqrt((D-1)*(D-2)))/D) + 3*x[1]*x[11]*((-D**2 + 3*D)/(2*D*np.sqrt(D-1))) -\
        (3/2)*x[1]*x[12]*(((D-2)*np.sqrt(D-1))/D)

    # G3: Exp Val for sum_{i,j,k} M_{ij} M_{jk} M_{ki}
    G3 = x[0]**3 + (x[1]**3)/(np.sqrt((D-1))) + 3*x[0]*(x[2] + (D-1)*x[6]) + 3*(x[1]/(np.sqrt(D-1)))*x[4] +\
        3*x[1]*(np.sqrt(D-1))*x[10] + 3*(x[1])*(np.sqrt(D-1))*x[6] + 3*x[1]*x[11]*((D*(D-3))/(2*(np.sqrt(D-1)))) -\
        3*x[1]*x[12]*(((D-2)*(np.sqrt(D-1)))/2)

    # G4: Exp Val for sum_{i,j,k} M_{ij} M_{jj} M_{jk}
    G4 = x[0]*x[2] + np.sqrt(D-1)*x[0]*x[3] +(D-1)*x[0]*x[5] + (D-1)*x[0]*x[6] +(D-1)*np.sqrt(D-2)*x[0]*x[7] + x[0]*x[2] +\
         (D-1)*x[0]*x[6] + np.sqrt(D-1)*x[1]*x[2] + ((D-1)**(3/2))*x[1]*x[6] + x[0]*x[2] + np.sqrt(D-1)*x[0]*x[3] +\
         (D-1)*x[0]*x[6] + x[0]*(D-1)*x[8] + (D-1)*np.sqrt(D-2)*x[0]*x[9] + (x[0]**3) + (x[0]**2)*x[1]*np.sqrt(D-1)

    # G5: Exp value for sum_{i,j,k,l} M_{ij}M_{kk}M_{ll}
    G5 = 2*(D*x[2] + D*(np.sqrt(D-1))*x[3])*(x[0] + np.sqrt(D-1)*x[1]) + (x[2] + (D-1)*x[4] +\
         2*np.sqrt(D-1)*x[3] + ((x[0] + np.sqrt(D-1)*x[1])**2)) * D*x[0]

    # G6: Exp value for sum_{i,j,k,l} M_{ij}M_{jk}M_{ll}
    G6 = 3*D*x[0]*x[2] + D*np.sqrt(D-1)*x[2]*x[1] + D*(D-1)*x[6]*x[0] + D*((D-1)**(3/2))*x[6]*x[1] +\
         2*D*(np.sqrt(D-1))*x[0]*x[3] + D*(x[0]**3) + D*np.sqrt(D-1)*(x[0]**2)*x[1]

    # G7: Exp val for sum_{i,j,k,l,m} M_{ij}M_{kl}M_{mm}
    G7 = ((D**2)*x[2])*(x[0] + np.sqrt(D-1)*x[1]) + 2*(D*x[2] + D*np.sqrt(D-1)*x[3])*D*x[0] + ((D*x[0])**2)*(x[0] +\
         np.sqrt(D-1)*x[1])

    # G8: Exp Val for sum_{i,j,k,l,m,n} M_{ij} M{kl} M{mn}
    G8 = 3*x[0]*(D**3)*x[2] + (x[0]**3)*(D**3)

    # G9: Exp Val for sum_{i_(1,2,3,4,5,6,7)} M_{i_(1,2)} M_{i_(3,4)} M_{i_(5,6)}M_{i_(7,7)}
    G9 = 3*((D**2)*x[2]*(D*x[2] + D*np.sqrt(D-1)*x[3])) + 3*((D**2)*x[2]*(D*x[0])*(x[0] + np.sqrt(D-1)*x[1]) +\
        (D*x[2] + D*np.sqrt(D-1)*(x[3]))*(D*x[0])**2) + ((D*x[0])**3)*(x[0] + np.sqrt(D-1)*x[1])

    # G10: Exp Val for sum_{i_(1,2,3,4,5,6,7,8)} M_{i_(1,2)} M_{i_(3,4)} M_{i_(5,6)}M_{i_(7,8)}
    G10 = 3*(D**4)*(x[2]**2) + 6*(D**4)*x[2]*(x[0]**2) + (D*x[0])**4
    
    #G11: Exp val M_ii^4
    G11 = ((D**2 - 2*D + 1)*x[1]**4 + x[0]**4 + 24*(D - 1)*x[3]*x[0]*x[1] + 12*(D - 1)*x[3]**2 + 6*(D - 1)*x[2]*x[4] + 3*(D**2 - 2*D + 1)*x[4]**2 + 3*(D**2 - 2*D + 1)*x[5]**2 + 12*(D**2 - 2*D + 1)*x[6]**2 + 12*(D**3 - 4*D**2 + 5*D - 2)*x[7]**2 + 3*(D**2 - 2*D + 1)*x[8]**2 + 24*(D**3 - 4*D**2 + 5*D - 2)*x[7]*x[9] + 12*(D**3 - 4*D**2 + 5*D - 2)*x[9]**2 + 3*(D**4 - 6*D**3 + 13*D**2 - 12*D + 4)*x[10]**2 + 6*((D - 1)*x[4] + (D - 1)*x[5] + 2*(D - 1)*x[6] + (D - 1)*x[8] + (D**2 - 3*D + 2)*x[10] + x[2])*x[0]**2 + 6*((D - 1)*x[0]**2 + (D - 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**2 - 2*D + 1)*x[5] + 2*(D**2 - 2*D + 1)*x[6] + (D**2 - 2*D + 1)*x[8] + (D**3 - 4*D**2 + 5*D - 2)*x[10])*x[1]**2 + 3*x[2]**2 + 6*((D - 1)*x[2] + (D**2 - 2*D + 1)*x[4])*x[5] + 12*((D - 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**2 - 2*D + 1)*x[5])*x[6] + 6*((D - 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**2 - 2*D + 1)*x[5] + 2*(D**2 - 2*D + 1)*x[6])*x[8] + 6*((D**2 - 3*D + 2)*x[2] + (D**3 - 4*D**2 + 5*D - 2)*x[4] + (D**3 - 4*D**2 + 5*D - 2)*x[5] + 2*(D**3 - 4*D**2 + 5*D - 2)*x[6] + (D**3 - 4*D**2 + 5*D - 2)*x[8])*x[10] + 4*((D - 1)*x[0]*x[1]**3 + 3*(D - 1)*x[3]*x[1]**2 + 3*(D - 1)*x[3]*x[4] + 3*(D - 1)*x[3]*x[5] + 6*(D - 1)*x[3]*x[6] + 3*(D - 1)*x[3]*x[8] + 3*(D**2 - 3*D + 2)*x[3]*x[10] + 3*x[3]*x[0]**2 + 3*x[2]*x[3] + (x[0]**3 + 3*((D - 1)*x[4] + (D - 1)*x[5] + 2*(D - 1)*x[6] + (D - 1)*x[8] + (D**2 - 3*D + 2)*x[10] + x[2])*x[0])*x[1] + 6*((D - 1)*x[3]*x[7] + (D - 1)*x[3]*x[9] + ((D - 1)*x[7] + (D - 1)*x[9])*x[0]*x[1])*np.sqrt(D - 2))*np.sqrt(D - 1) + 12*((D**2 - 2*D + 1)*x[7]*x[8] + ((D - 1)*x[7] + (D - 1)*x[9])*x[0]**2 + ((D**2 - 2*D + 1)*x[7] + (D**2 - 2*D + 1)*x[9])*x[1]**2 + ((D - 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**2 - 2*D + 1)*x[5] + 2*(D**2 - 2*D + 1)*x[6])*x[7] + ((D - 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**2 - 2*D + 1)*x[5] + 2*(D**2 - 2*D + 1)*x[6] + (D**2 - 2*D + 1)*x[8])*x[9] + ((D**3 - 4*D**2 + 5*D - 2)*x[7] + (D**3 - 4*D**2 + 5*D - 2)*x[9])*x[10])*np.sqrt(D - 2))/D**3
    
    #G12: Exp val M_ij^4
    G12 = 1/4*(4*(D - 1)*x[0]**4 + 4*(D**2 - 3*D + 3)*x[1]**4 + 96*(D - 1)*x[3]*x[0]*x[1] + 12*(D - 1)*x[2]**2 + 48*(D - 1)*x[3]**2 + 24*(D - 1)*x[2]*x[4] + 12*(D**2 - 3*D + 3)*x[4]**2 + 12*(D**3 - 3*D**2 + 3*D - 1)*x[5]**2 + 48*(D**2 - 3*D + 2)*x[4]*x[6] + 48*(D**2 - 2*D + 1)*x[6]**2 + 48*(D**3 - 4*D**2 + 5*D - 2)*x[7]**2 + 12*(D**3 - 3*D**2 + 3*D - 1)*x[8]**2 + 96*(D**3 - 4*D**2 + 5*D - 2)*x[7]*x[9] + 48*(D**3 - 4*D**2 + 5*D - 2)*x[9]**2 + 12*(D**4 - 7*D**3 + 19*D**2 - 21*D + 8)*x[10]**2 + 3*(D**5 - 6*D**4 + 9*D**3)*x[11]**2 + 3*(D**5 - 6*D**4 + 13*D**3 - 12*D**2 + 4*D)*x[12]**2 + 12*(2*(D - 1)*x[2] + 2*(D - 1)*x[4] + 2*(D**2 - 2*D + 1)*x[5] + 2*(D**2 - 2*D + 1)*x[8] + 2*(D**2 - 2*D + 1)*x[10] + (D**3 - 4*D**2 + 3*D)*x[11] + (D**3 - 4*D**2 + 5*D - 2)*x[12])*x[0]**2 + 12*(2*(D - 1)*x[0]**2 + 2*(D - 1)*x[2] + 2*(D**2 - 3*D + 3)*x[4] + 2*(D**2 - 2*D + 1)*x[5] + 4*(D**2 - 3*D + 2)*x[6] + 2*(D**2 - 2*D + 1)*x[8] + 2*(D**3 - 5*D**2 + 9*D - 5)*x[10] + (D**2 - 3*D)*x[11] + (D**2 - 3*D + 2)*x[12])*x[1]**2 + 24*((D**2 - 2*D + 1)*x[2] + (D**2 - 2*D + 1)*x[4])*x[5] + 24*((D**2 - 2*D + 1)*x[2] + (D**2 - 2*D + 1)*x[4] + (D**3 - 3*D**2 + 3*D - 1)*x[5])*x[8] + 24*((D**2 - 2*D + 1)*x[2] + (D**3 - 5*D**2 + 9*D - 5)*x[4] + (D**3 - 3*D**2 + 3*D - 1)*x[5] + 2*(D**3 - 5*D**2 + 7*D - 3)*x[6] + (D**3 - 3*D**2 + 3*D - 1)*x[8])*x[10] + 12*((D**3 - 4*D**2 + 3*D)*x[2] + (D**2 - 3*D)*x[4] + (D**4 - 5*D**3 + 7*D**2 - 3*D)*x[5] - 2*(D**3 - 4*D**2 + 3*D)*x[6] + (D**4 - 5*D**3 + 7*D**2 - 3*D)*x[8] + 2*(D**3 - 4*D**2 + 3*D)*x[10])*x[11] + 6*(2*(D**3 - 4*D**2 + 5*D - 2)*x[2] + 2*(D**2 - 3*D + 2)*x[4] + 2*(D**4 - 5*D**3 + 9*D**2 - 7*D + 2)*x[5] - 4*(D**3 - 4*D**2 + 5*D - 2)*x[6] + 2*(D**4 - 5*D**3 + 9*D**2 - 7*D + 2)*x[8] + 4*(D**3 - 4*D**2 + 5*D - 2)*x[10] + (D**5 - 6*D**4 + 11*D**3 - 6*D**2)*x[11])*x[12] + 8*(2*(D - 2)*x[0]*x[1]**3 + 6*(D - 2)*x[3]*x[1]**2 + 6*(D - 2)*x[3]*x[4] + 12*(D - 1)*x[3]*x[6] + 6*(D**2 - 4*D + 3)*x[3]*x[10] - 3*(D**2 - 3*D)*x[3]*x[11] - 3*(D**2 - 3*D + 2)*x[3]*x[12] + 3*(2*(D - 2)*x[4] + 4*(D - 1)*x[6] + 2*(D**2 - 4*D + 3)*x[10] - (D**2 - 3*D)*x[11] - (D**2 - 3*D + 2)*x[12])*x[0]*x[1] + 12*((D - 1)*x[3]*x[7] + (D - 1)*x[3]*x[9] + ((D - 1)*x[7] + (D - 1)*x[9])*x[0]*x[1])*np.sqrt(D - 2))*np.sqrt(D - 1) + 24*(2*((D**2 - 3*D + 2)*x[7] + (D**2 - 3*D + 2)*x[9])*x[1]**2 + 2*((D**2 - 3*D + 2)*x[4] + 2*(D**2 - 2*D + 1)*x[6])*x[7] + 2*((D**2 - 3*D + 2)*x[4] + 2*(D**2 - 2*D + 1)*x[6])*x[9] + 2*((D**3 - 5*D**2 + 7*D - 3)*x[7] + (D**3 - 5*D**2 + 7*D - 3)*x[9])*x[10] - ((D**3 - 4*D**2 + 3*D)*x[7] + (D**3 - 4*D**2 + 3*D)*x[9])*x[11] - ((D**3 - 4*D**2 + 5*D - 2)*x[7] + (D**3 - 4*D**2 + 5*D - 2)*x[9])*x[12])*np.sqrt(D - 2))/(D**3 - D**2)
    
    #G13: Exp val M_ij M_jk M_pq M_qr
    G13 = D**2*x[0]**4 + 3*D**2*x[2]**2 + 2*(D**3 - D**2)*x[2]*x[6] + (D**4 - D**3)*x[6]**2 + (D**3 - D**2)*x[5]*x[8] + 2*(3*D**2*x[2] + (D**3 - D**2)*x[6])*x[0]**2
    
    #G14: Exp val M_ij M_jk M_kl
    G14 = ((D**2 - D)*x[6]*x[1] + (D*x[0]**3 + (3*D*x[2] + 2*(D**2 - D)*x[6])*x[0])*np.sqrt(D - 1))/np.sqrt(D - 1)
    
    #G15: Exp val M_ij M_jk M_kl M_lm
    G15 = D*x[0]**4 + D*x[6]*x[1]**2 + 3*D*x[2]**2 + (D**3 - D**2)*x[6]**2 + (D**2 - D)*x[5]*x[8] + (D**2 - D)*x[7]*x[9] + (D**2 - D)*x[6]*x[10] + 1/2*(D**3 - 3*D**2)*x[6]*x[11] - 1/2*(D**3 - 3*D**2 + 2*D)*x[6]*x[12] + 3*(2*D*x[2] + (D**2 - D)*x[6])*x[0]**2 + (3*(D**2 - D)*x[2] + D*x[4])*x[6] + 2*(D*x[6]*x[0]*x[1] + D*x[3]*x[6])*np.sqrt(D - 1)    
    return(np.array([G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15]))

In [68]:
thmean5 = theory_expectations2(x1,100)
thmean6 = theory_expectations2(x2,100)
thmean7 = theory_expectations2(x3,100)
thmean8 = theory_expectations2(x4,100)
thmean9 = theory_expectations2(x5,100)
thmean10 = theory_expectations2(x6,100)

In [10]:
print(thmean1)
thmean5

[-9.05606710e-04 -6.42257337e-03 -1.23161781e-03 -3.38344968e-03
 -4.63670332e-01 -6.86288146e-02 -4.48764985e+00 -6.66418252e+01
  3.10016653e+01  7.13146405e+02]


array([-9.05606710e-04, -6.42257337e-03, -1.23161781e-03, -3.38344968e-03,
       -4.63670332e-01, -6.86288146e-02, -4.48764985e+00, -6.66418252e+01,
        3.10016653e+01,  7.13146405e+02,  5.96056128e-04,  5.25879000e-02,
        1.85236880e+00, -1.04376576e-02,  1.89310386e-02])

# linear (2) + quadratic (11) + cubic (8) + quartic(2)  (23 total)

In [71]:
# Make observables object
set2 = mf.observables(["Linear.txt","Quadratic.txt","Cubic1.txt","Quartic1.txt"])
# Load matrices and make vector dictionaries
objsubset2 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set2)
subobjset2 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set2,[0])
obj08sub02set2 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set2,[1])
obj09sub01set2 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set2,[1])
obj05sub05set2 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.5,0.5]],set2,[1])
obj02sub08set2 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set2,[1])

In [72]:
#Store the experimental values for later use, both of the mean and std
objsubset2exp = [objsubset2.mean.copy(),objsubset2.std.copy()]
subobjset2exp = [subobjset2.mean.copy(),subobjset2.std.copy()]
obj08sub02set2exp = [obj08sub02set2.mean.copy(),obj08sub02set2.std.copy()]
obj09sub01set2exp = [obj09sub01set2.mean.copy(),obj09sub01set2.std.copy()]
obj05sub05set2exp = [obj05sub05set2.mean.copy(),obj05sub05set2.std.copy()]
obj02sub08set2exp = [obj02sub08set2.mean.copy(),obj02sub08set2.std.copy()]

In [73]:
#Reassign the means stored in the vector dictionary to be the theoretical means
objsubset2.mean = np.concatenate((objsubset1.mean,thmean1),axis=0)
obj09sub01set2.mean = np.concatenate((obj09sub01set1.mean,thmean2),axis=0)
obj08sub02set2.mean = np.concatenate((obj08sub02set1.mean,thmean3),axis=0)
obj05sub05set2.mean = np.concatenate((obj05sub05set1.mean,thmean4a),axis=0)
obj02sub08set2.mean = np.concatenate((obj02sub08set1.mean,thmean4b),axis=0)
subobjset2.mean = np.concatenate((subobjset1.mean,thmean4),axis=0)

In [74]:
#Reassign the std to be the theoretical std, the total number of verbs is 1160
objsubset2.std = np.sqrt(np.sum([(a.val - objsubset2.mean) ** 2 for a in objsubset2.values()],axis=0)/1160)
obj09sub01set2.std = np.sqrt(np.sum([(a.val - obj09sub01set2.mean) ** 2 for a in obj09sub01set2.values()],axis=0)/1160)
obj08sub02set2.std = np.sqrt(np.sum([(a.val - obj08sub02set2.mean) ** 2 for a in obj08sub02set2.values()],axis=0)/1160)
obj05sub05set2.std = np.sqrt(np.sum([(a.val - obj05sub05set2.mean) ** 2 for a in obj05sub05set2.values()],axis=0)/1160)
obj02sub08set2.std = np.sqrt(np.sum([(a.val - obj02sub08set2.mean) ** 2 for a in obj02sub08set2.values()],axis=0)/1160)
subobjset2.std = np.sqrt(np.sum([(a.val - subobjset2.mean) ** 2 for a in subobjset2.values()],axis=0)/1160)

In [75]:
#And finally the mean squared
objsubset2.mean_sqr = np.mean(np.array([ a.val for a in objsubset2.values()]) ** 2, axis = 0)
obj09sub01set2.mean_sqr = np.mean(np.array([ a.val for a in obj09sub01set2.values()]) ** 2, axis = 0)
obj08sub02set2.mean_sqr = np.mean(np.array([ a.val for a in obj08sub02set2.values()]) ** 2, axis = 0)
obj05sub05set2.mean_sqr = np.mean(np.array([ a.val for a in obj05sub05set2.values()]) ** 2, axis = 0)
obj02sub08set2.mean_sqr = np.mean(np.array([ a.val for a in obj02sub08set2.values()]) ** 2, axis = 0)
subobjset2.mean_sqr = np.mean(np.array([ a.val for a in subobjset2.values()]) ** 2, axis = 0)

### Relation type NONE

#### Usual scalar product

In [73]:
#observable vectors
print(mf.averaged_product(simdict,'NONE',objsubset2,"mean"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set2,"mean"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set2,"mean"))
print(mf.averaged_product(simdict,'NONE',subobjset2,"mean"))

[0.39696668847498423, 0.2969469676985821]
[0.3879921373411235, 0.30183395144796515]
[0.3755981685402756, 0.3104361079512253]
[0.26105800929798306, 0.3820393951225187]


In [98]:
# observable vector deviations
print(mf.averaged_product(simdict,'NONE',objsubset2,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set2,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set2,"std_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset2,"std_dev"))

[0.16342154446757987, 0.5012004843230823]
[0.165061650904801, 0.4993422499433231]
[0.1704649355135085, 0.49749007372948817]
[0.18758704579993235, 0.5150564543195832]


#### Plain scalar product

In [70]:
print(mf.averaged_product(simdict,'NONE',objsubset2,"plain"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set2,"plain"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set2,"plain"))
print(mf.averaged_product(simdict,'NONE',subobjset2,"plain"))

[0.5952026614624384, 0.34852163591252405]
[0.6030895353704414, 0.33788323371837975]
[0.6090586043702529, 0.32907843824632604]
[0.5888949739478951, 0.3460490031268164]


In [57]:
print(mf.averaged_product(simdict,'NONE',objsubset2,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set2,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set2,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset2,"plain_dev"))

[0.4068257497088816, 0.8706740310642554]
[0.38236144926001486, 0.8692425976043144]
[0.395196329667925, 0.8420052633601564]
[0.5373006330499959, 0.824895745944837]


### Relation type SYNONYMS

#### Usual scalar product

In [74]:
#observable vectors
print(mf.averaged_product(simdict,'SYNONYMS',objsubset2,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set2,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set2,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset2,"mean"))

[0.4424508774636527, 0.2552672771344092]
[0.4374229455648321, 0.25389426967226736]
[0.4317166132419034, 0.2589245526504496]
[0.32905737096885584, 0.3856356523731884]


In [58]:
# observable vector deviations
print(mf.averaged_product(simdict,'SYNONYMS',objsubset2,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set2,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set2,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset2,"std_dev"))

[0.2795145107377657, 0.4776719926134788]
[0.2845206572927047, 0.4739889394742949]
[0.2934521604205345, 0.4704812326382969]
[0.3058092834487867, 0.48062087590046754]


#### Plain scalar product

In [22]:
print(mf.averaged_product(simdict,'SYNONYMS',objsubset2,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set2,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set2,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset2,"plain"))

[0.6004330990559624, 0.35459868530884353]
[0.6027001314807311, 0.3438758230415265]
[0.6077537891933465, 0.3316370387169906]
[0.6175030504037731, 0.3523609111981688]


In [59]:
print(mf.averaged_product(simdict,'SYNONYMS',objsubset2,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set2,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set2,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset2,"plain_dev"))

[0.3728864231906694, 0.8830543145968975]
[0.3613770305380966, 0.877425025867639]
[0.4051273929912112, 0.836030971041063]
[0.5363017518206178, 0.8292151717464932]


### Relation type ANTONYMS

#### Usual scalar product

In [24]:
#observable vectors
print(mf.averaged_product(simdict,'ANTONYMS',objsubset2,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set2,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set2,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset2,"mean"))

[0.39320135940155854, 0.25983143147885057]
[0.3819479371601253, 0.2646080904487618]
[0.37646549029039683, 0.26874667735786895]
[0.34280148390816295, 0.32258068717829597]


In [99]:
# observable vector deviations
print(mf.averaged_product(simdict,'ANTONYMS',objsubset2,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set2,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set2,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset2,"std_dev"))

[0.08947500811193136, 0.4425619356250345]
[0.08415140928010593, 0.44447042006834886]
[0.08969236900972732, 0.4459821274615428]
[0.13835398710295493, 0.4528896305032765]


#### Plain scalar product

In [26]:
print(mf.averaged_product(simdict,'ANTONYMS',objsubset2,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set2,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set2,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset2,"plain"))

[0.6485978187026449, 0.3686897109418542]
[0.6026751274535302, 0.37677061918815874]
[0.5622340803343061, 0.3658868698097848]
[0.6659679544848128, 0.3747666924729258]


In [61]:
print(mf.averaged_product(simdict,'ANTONYMS',objsubset2,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set2,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set2,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset2,"plain_dev"))

[0.23816749425684763, 0.9196932648602609]
[0.23634372380852123, 0.8887760076986267]
[0.22715155605023007, 0.8693610356095384]
[0.483465437938861, 0.8558718896948091]


### Relation type HYPER/HYPONYMS

#### Usual scalar product

In [28]:
#observable vectors
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',objsubset2,"mean"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj09sub01set2,"mean"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj08sub02set2,"mean"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',subobjset2,"mean"))

[0.4249294683633869, 0.26617174312424186]
[0.41879805136318593, 0.27244109906499825]
[0.41260697959078174, 0.281914952529803]
[0.3181464480028766, 0.3455584144825405]


In [62]:
# observable vector deviations
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',objsubset2,"std_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj09sub01set2,"std_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj08sub02set2,"std_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',subobjset2,"std_dev"))

[0.12606643086638844, 0.46956328205535547]
[0.12799753440154726, 0.46612867607494807]
[0.1336009029785951, 0.46348591707649867]
[0.1278878876143317, 0.4731739352259034]


#### Plain scalar product

In [30]:
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',objsubset2,"plain"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj09sub01set2,"plain"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj08sub02set2,"plain"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',subobjset2,"plain"))

[0.5934271549010992, 0.35449810612409416]
[0.5891098591583731, 0.34672783533221907]
[0.5952341291057545, 0.33768849913848326]
[0.5898688485837195, 0.3585830041320534]


In [63]:
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',objsubset2,"plain_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj09sub01set2,"plain_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',obj08sub02set2,"plain_dev"))
print(mf.averaged_product(simdict,'HYPER/HYPONYMS',subobjset2,"plain_dev"))

[0.31966868209468374, 0.904335461635918]
[0.3198362685384297, 0.8882843451830617]
[0.3746020734291855, 0.8402617024110218]
[0.4406718223940622, 0.8787677861569736]


### Relation type COHYPONYMS

#### Usual scalar product

In [32]:
#observable vectors
print(mf.averaged_product(simdict,'COHYPONYMS',objsubset2,"mean"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj09sub01set2,"mean"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj08sub02set2,"mean"))
print(mf.averaged_product(simdict,'COHYPONYMS',subobjset2,"mean"))

[0.41485555780992456, 0.2762073969495413]
[0.4104442440127617, 0.27496684133333466]
[0.3991823082561555, 0.2796427783606923]
[0.3244571339171163, 0.38158473699390255]


In [64]:
# observable vector deviations
print(mf.averaged_product(simdict,'COHYPONYMS',objsubset2,"std_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj09sub01set2,"std_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj08sub02set2,"std_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',subobjset2,"std_dev"))

[0.18474741375766923, 0.4587772520978451]
[0.18942444014611484, 0.45121741457914566]
[0.1965467364042946, 0.44854202743876465]
[0.24068229708157465, 0.4839543194179557]


#### Plain scalar product

In [34]:
print(mf.averaged_product(simdict,'COHYPONYMS',objsubset2,"plain"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj09sub01set2,"plain"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj08sub02set2,"plain"))
print(mf.averaged_product(simdict,'COHYPONYMS',subobjset2,"plain"))

[0.6124159201993651, 0.34774248174389055]
[0.6085176858876239, 0.3447513707722667]
[0.6088116610270333, 0.32897971618113503]
[0.6065613829025644, 0.35253911100651497]


In [65]:
print(mf.averaged_product(simdict,'COHYPONYMS',objsubset2,"plain_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj09sub01set2,"plain_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',obj08sub02set2,"plain_dev"))
print(mf.averaged_product(simdict,'COHYPONYMS',subobjset2,"plain_dev"))

[0.3926373538652845, 0.8871248181186109]
[0.41464661996240143, 0.8636726703977783]
[0.42698835833991, 0.8265279794248281]
[0.581318123959344, 0.7936982976995525]


# Cubic (8) + quartic(2) (10 total)

In [60]:
# Make observables object
set3 = mf.observables(["Cubic1.txt","Quartic1.txt"])
# Load matrices and make vector dictionaries
objsubset3 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set3)
subobjset3 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set3,[0])
obj08sub02set3 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set3,[1])
obj09sub01set3 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set3,[1])
obj05sub05set3 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.5,0.5]],set3,[1])
obj02sub08set3 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set3,[1])

In [61]:
#Store the experimental values for later use, both of the mean and std
objsubset3exp = [objsubset3.mean.copy(),objsubset3.std.copy()]
subobjset3exp = [subobjset3.mean.copy(),subobjset3.std.copy()]
obj08sub02set3exp = [obj08sub02set3.mean.copy(),obj08sub02set3.std.copy()]
obj09sub01set3exp = [obj09sub01set3.mean.copy(),obj09sub01set3.std.copy()]
obj05sub05set3exp = [obj05sub05set3.mean.copy(),obj05sub05set3.std.copy()]
obj02sub08set3exp = [obj02sub08set3.mean.copy(),obj02sub08set3.std.copy()]

In [69]:
#Reassign the means stored in the vector dictionary to be the theoretical means
objsubset3.mean = thmean1
obj09sub01set3.mean = thmean2
obj08sub02set3.mean = thmean3
obj05sub05set3.mean = thmean4a
obj02sub08set3.mean = thmean4b
subobjset3.mean = thmean4

In [70]:
#Reassign the std to be the theoretical std, the total number of verbs is 1160
objsubset3.std = np.sqrt(np.sum([(a.val - objsubset3.mean) ** 2 for a in objsubset3.values()],axis=0)/1160)
obj09sub01set3.std = np.sqrt(np.sum([(a.val - obj09sub01set3.mean) ** 2 for a in obj09sub01set3.values()],axis=0)/1160)
obj08sub02set3.std = np.sqrt(np.sum([(a.val - obj08sub02set3.mean) ** 2 for a in obj08sub02set3.values()],axis=0)/1160)
obj05sub05set3.std = np.sqrt(np.sum([(a.val - obj05sub05set3.mean) ** 2 for a in obj05sub05set3.values()],axis=0)/1160)
obj02sub08set3.std = np.sqrt(np.sum([(a.val - obj02sub08set3.mean) ** 2 for a in obj02sub08set3.values()],axis=0)/1160)
subobjset3.std = np.sqrt(np.sum([(a.val - subobjset3.mean) ** 2 for a in subobjset3.values()],axis=0)/1160)

### Relation type NONE

#### Usual scalar product

In [37]:
#observable vectors
print(mf.averaged_product(simdict,'NONE',objsubset3,"mean"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set3,"mean"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set3,"mean"))
print(mf.averaged_product(simdict,'NONE',subobjset3,"mean"))

[0.11062917742952184, 0.4167074322316934]
[0.10708315389808418, 0.41140266025866506]
[0.10390558945427522, 0.4099046803328103]
[0.01984377095673164, 0.5563609657614086]


In [77]:
# observable vector deviations
print(mf.averaged_product(simdict,'NONE',objsubset3,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set3,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set3,"std_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset3,"std_dev"))

[0.17998843612296261, 0.5137827642037696]
[0.17604111587280993, 0.5059344436178395]
[0.16887458363159644, 0.4955873884983094]
[0.22536402489428037, 0.529373261707615]


#### Plain scalar product

In [39]:
print(mf.averaged_product(simdict,'NONE',objsubset3,"plain"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set3,"plain"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set3,"plain"))
print(mf.averaged_product(simdict,'NONE',subobjset3,"plain"))

[0.5519718924199511, 0.41881326367872923]
[0.5335069633731999, 0.4254152815843077]
[0.5076533379870708, 0.43760110995429885]
[0.554855675383644, 0.4065366954299626]


In [78]:
print(mf.averaged_product(simdict,'NONE',objsubset3,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set3,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set3,"plain_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset3,"plain_dev"))

[0.4075942303253251, 0.8735512883521559]
[0.3834964135969046, 0.8751643997712393]
[0.39850227435275604, 0.8519357656734695]
[0.5376780750510143, 0.8253751812458164]


### Relation type SYNONYMS

#### Usual scalar product

In [41]:
#observable vectors
print(mf.averaged_product(simdict,'SYNONYMS',objsubset3,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set3,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set3,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset3,"mean"))

[0.12918037897319296, 0.3985113060682579]
[0.13017503488431148, 0.4024251498591181]
[0.13657147208338527, 0.4043336121274284]
[0.023927742072425408, 0.5218317723681576]


In [79]:
# observable vector deviations
print(mf.averaged_product(simdict,'SYNONYMS',objsubset3,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set3,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set3,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset3,"std_dev"))

[0.26077379359520714, 0.505324436944525]
[0.2624725549484761, 0.49540591757182145]
[0.2616326990157913, 0.4844628082587479]
[0.27232920176431524, 0.5036461026646174]


#### Plain scalar product

In [43]:
print(mf.averaged_product(simdict,'SYNONYMS',objsubset3,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set3,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set3,"plain"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset3,"plain"))

[0.5537120723933104, 0.4478156796132517]
[0.5096196323151625, 0.4682167695952224]
[0.4927181970759464, 0.46387298196245585]
[0.5673760273470484, 0.4312207329575683]


In [80]:
print(mf.averaged_product(simdict,'SYNONYMS',objsubset3,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set3,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set3,"plain_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset3,"plain_dev"))

[0.37272729318292636, 0.886241272530502]
[0.3586716935918036, 0.8890551535076976]
[0.4047210602128136, 0.8513214122277357]
[0.5363336553170703, 0.8296938053593249]


### Relation type ANTONYMS

#### Usual scalar product

In [45]:
#observable vectors
print(mf.averaged_product(simdict,'ANTONYMS',objsubset3,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set3,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set3,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset3,"mean"))

[0.056065586218732644, 0.42090342009589515]
[0.05804310832889628, 0.42155423321186336]
[0.05596848698937469, 0.41191548640572656]
[0.06456161483964938, 0.5626718836744539]


In [81]:
# observable vector deviations
print(mf.averaged_product(simdict,'ANTONYMS',objsubset3,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set3,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set3,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset3,"std_dev"))

[0.053465442995226174, 0.45689298888200375]
[0.06369498308592027, 0.4548418691906052]
[0.06697394688295824, 0.440264873207358]
[0.11112158739691015, 0.5345817591057888]


#### Plain scalar product

In [47]:
print(mf.averaged_product(simdict,'ANTONYMS',objsubset3,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set3,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set3,"plain"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset3,"plain"))

[0.666551221655536, 0.36288405692790765]
[0.6198273941447612, 0.3854351648996523]
[0.5456408478021069, 0.42350989562208907]
[0.6540402362090085, 0.39148848460956964]


In [82]:
print(mf.averaged_product(simdict,'ANTONYMS',objsubset3,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set3,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set3,"plain_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset3,"plain_dev"))

[0.23889645510722277, 0.9246943535031679]
[0.23786537902503757, 0.9032536794819473]
[0.22366026735499603, 0.8914462211626979]
[0.48376089131647176, 0.8565157513872993]


# linear (2) + quadratic (11) + cubic (8) + quartic(2) +additional(5) (28 total)

In [55]:
# Make observables object
set4 = mf.observables(["Linear.txt","Quadratic.txt","Cubic1.txt","Quartic1.txt","Additional1.txt"])
# Load matrices and make vector dictionaries
objsubset4 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set4)
subobjset4 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set4,[0])
obj08sub02set4 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set4,[1])
obj09sub01set4 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set4,[1])
obj05sub05set4 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.5,0.5]],set4,[1])
obj02sub08set4 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set4,[1])

In [56]:
#Store the experimental values for later use, both of the mean and std
objsubset4exp = [objsubset4.mean.copy(),objsubset4.std.copy()]
subobjset4exp = [subobjset4.mean.copy(),subobjset4.std.copy()]
obj08sub02set4exp = [obj08sub02set4.mean.copy(),obj08sub02set4.std.copy()]
obj09sub01set4exp = [obj09sub01set4.mean.copy(),obj09sub01set4.std.copy()]
obj05sub05set4exp = [obj05sub05set4.mean.copy(),obj05sub05set4.std.copy()]
obj02sub08set4exp = [obj02sub08set4.mean.copy(),obj02sub08set4.std.copy()]

In [57]:
#Reassign the means stored in the vector dictionary to be the theoretical means
objsubset4.mean = np.concatenate((objsubset1.mean,thmean5),axis=0)
obj09sub01set4.mean = np.concatenate((obj09sub01set1.mean,thmean6),axis=0)
obj08sub02set4.mean = np.concatenate((obj08sub02set1.mean,thmean7),axis=0)
obj05sub05set4.mean = np.concatenate((obj05sub05set1.mean,thmean9),axis=0)
obj02sub08set4.mean = np.concatenate((obj02sub08set1.mean,thmean10),axis=0)
subobjset4.mean = np.concatenate((subobjset1.mean,thmean8),axis=0)

In [58]:
#Reassign the std to be the theoretical std, the total number of verbs is 1160
objsubset4.std = np.sqrt(np.sum([(a.val - objsubset4.mean) ** 2 for a in objsubset4.values()],axis=0)/1160)
obj09sub01set4.std = np.sqrt(np.sum([(a.val - obj09sub01set4.mean) ** 2 for a in obj09sub01set4.values()],axis=0)/1160)
obj08sub02set4.std = np.sqrt(np.sum([(a.val - obj08sub02set4.mean) ** 2 for a in obj08sub02set4.values()],axis=0)/1160)
obj05sub05set4.std = np.sqrt(np.sum([(a.val - obj05sub05set4.mean) ** 2 for a in obj05sub05set4.values()],axis=0)/1160)
obj02sub08set4.std = np.sqrt(np.sum([(a.val - obj02sub08set4.mean) ** 2 for a in obj02sub08set4.values()],axis=0)/1160)
subobjset4.std = np.sqrt(np.sum([(a.val - subobjset4.mean) ** 2 for a in subobjset4.values()],axis=0)/1160)

In [59]:
#And finally the mean squared
objsubset4.mean_sqr = np.mean(np.array([ a.val for a in objsubset4.values()]) ** 2, axis = 0)
obj09sub01set4.mean_sqr = np.mean(np.array([ a.val for a in obj09sub01set4.values()]) ** 2, axis = 0)
obj08sub02set4.mean_sqr = np.mean(np.array([ a.val for a in obj08sub02set4.values()]) ** 2, axis = 0)
obj05sub05set4.mean_sqr = np.mean(np.array([ a.val for a in obj05sub05set4.values()]) ** 2, axis = 0)
obj02sub08set4.mean_sqr = np.mean(np.array([ a.val for a in obj02sub08set4.values()]) ** 2, axis = 0)
subobjset4.mean_sqr = np.mean(np.array([ a.val for a in subobjset4.values()]) ** 2, axis = 0)

### Relation type NONE

#### Usual scalar product

In [16]:
#observable vectors
print(mf.averaged_product(simdict,'NONE',objsubset4,"mean"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set4,"mean"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set4,"mean"))
print(mf.averaged_product(simdict,'NONE',subobjset4,"mean"))

[0.3826756265207932, 0.29113978558119225]
[0.3770090762099168, 0.29699665525632674]
[0.3656016046609286, 0.3055564139001798]
[0.25664911899247356, 0.37940873424947624]


In [17]:
# observable vector deviations
print(mf.averaged_product(simdict,'NONE',objsubset4,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set4,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set4,"std_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset4,"std_dev"))

[0.16710471272492586, 0.4897062417383025]
[0.1686572539026655, 0.48994492043460686]
[0.17358665064083356, 0.4886746917157874]
[0.18928004690236228, 0.5104248827015043]


### Relation type SYNONYMS

#### Usual scalar product

In [18]:
#observable vectors
print(mf.averaged_product(simdict,'SYNONYMS',objsubset4,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set4,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set4,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset4,"mean"))

[0.4224202749299982, 0.24907052050389267]
[0.42258085557474656, 0.24864292874909247]
[0.42109797378083624, 0.2540479274175125]
[0.32188104478733953, 0.381958649409863]


In [19]:
# observable vector deviations
print(mf.averaged_product(simdict,'SYNONYMS',objsubset4,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set4,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set4,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset4,"std_dev"))

[0.2751931455081116, 0.4675157793020683]
[0.2822894226424081, 0.46528076044191563]
[0.29398182325132916, 0.46192499598847825]
[0.30245540076292815, 0.47888045634359644]


### Relation type ANTONYMS

#### Usual scalar product

In [20]:
#observable vectors
print(mf.averaged_product(simdict,'ANTONYMS',objsubset4,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set4,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set4,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset4,"mean"))

[0.36772265559729966, 0.2504211696716251]
[0.3637703147318423, 0.2563760368706155]
[0.3659668138072767, 0.26316577541384467]
[0.3355258607515912, 0.3188716057157105]


In [21]:
# observable vector deviations
print(mf.averaged_product(simdict,'ANTONYMS',objsubset4,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set4,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set4,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset4,"std_dev"))

[0.08416508709355439, 0.41904813625573994]
[0.0817447307519454, 0.4264960493748427]
[0.09236254972640569, 0.433827654561518]
[0.1379490720164012, 0.4466951587693989]


# Cubic (8) + quartic(2) + additional(5) (15 total)

In [14]:
# Make observables object
set5 = mf.observables(["Cubic1.txt","Quartic1.txt","Additional1.txt"])
# Load matrices and make vector dictionaries
objsubset5 = mf.vector_dictionary("matrices_1160_arg_obj_context_subj.txt",set5)
subobjset5 = mf.vector_dictionary("matrices_1160_arg_subj_context_obj.txt",set5,[0])
obj08sub02set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.8,0.2]],set5,[1])
obj09sub01set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.9,0.1]],set5,[1])
obj05sub05set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.5,0.5]],set5,[1])
obj02sub08set5 = mf.vector_dictionary([["matrices_1160_arg_obj_context_subj.txt","matrices_1160_arg_subj_context_obj.txt"],[0.2,0.8]],set5,[1])

In [15]:
#Store the experimental values for later use, both of the mean and std
objsubset5exp = [objsubset5.mean.copy(),objsubset5.std.copy()]
subobjset5exp = [subobjset5.mean.copy(),subobjset5.std.copy()]
obj08sub02set5exp = [obj08sub02set5.mean.copy(),obj08sub02set5.std.copy()]
obj09sub01set5exp = [obj09sub01set5.mean.copy(),obj09sub01set5.std.copy()]
obj05sub05set5exp = [obj05sub05set5.mean.copy(),obj05sub05set5.std.copy()]
obj02sub08set5exp = [obj02sub08set5.mean.copy(),obj02sub08set5.std.copy()]

In [18]:
#Reassign the means stored in the vector dictionary to be the theoretical means
objsubset5.mean = thmean5
obj09sub01set5.mean = thmean6
obj08sub02set5.mean = thmean7
subobjset5.mean = thmean8
obj05sub05set5.mean = thmean9
obj02sub08set5.mean = thmean10

In [19]:
#Reassign the std to be the theoretical std, the total number of verbs is 1160
objsubset5.std = np.sqrt(np.sum([(a.val - objsubset5.mean) ** 2 for a in objsubset5.values()],axis=0)/1160)
obj09sub01set5.std = np.sqrt(np.sum([(a.val - obj09sub01set5.mean) ** 2 for a in obj09sub01set5.values()],axis=0)/1160)
obj08sub02set5.std = np.sqrt(np.sum([(a.val - obj08sub02set5.mean) ** 2 for a in obj08sub02set5.values()],axis=0)/1160)
subobjset5.std = np.sqrt(np.sum([(a.val - subobjset5.mean) ** 2 for a in subobjset5.values()],axis=0)/1160)
obj05sub05set5.std = np.sqrt(np.sum([(a.val - obj05sub05set5.mean) ** 2 for a in obj05sub05set5.values()],axis=0)/1160)
obj02sub08set5.std = np.sqrt(np.sum([(a.val - obj02sub08set5.mean) ** 2 for a in obj02sub08set5.values()],axis=0)/1160)

In [20]:
#And finally the mean squared
objsubset5.mean_sqr = np.mean(np.array([ a.val for a in objsubset5.values()]) ** 2, axis = 0)
obj09sub01set5.mean_sqr = np.mean(np.array([ a.val for a in obj09sub01set5.values()]) ** 2, axis = 0)
obj08sub02set5.mean_sqr = np.mean(np.array([ a.val for a in obj08sub02set5.values()]) ** 2, axis = 0)
subobjset5.mean_sqr = np.mean(np.array([ a.val for a in subobjset5.values()]) ** 2, axis = 0)
obj05sub05set5.mean_sqr = np.mean(np.array([ a.val for a in obj05sub05set5.values()]) ** 2, axis = 0)
obj02sub08set5.mean_sqr = np.mean(np.array([ a.val for a in obj02sub08set5.values()]) ** 2, axis = 0)

### Relation type NONE

#### Usual scalar product

In [27]:
#observable vectors
print(mf.averaged_product(simdict,'NONE',objsubset5,"mean"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set5,"mean"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set5,"mean"))
print(mf.averaged_product(simdict,'NONE',subobjset5,"mean"))

[0.12618448633210183, 0.3520788741228348]
[0.12928700209558058, 0.3506145424305712]
[0.12903551560389923, 0.34679260573998394]
[0.03592779507084502, 0.4632461094693515]


In [28]:
# observable vector deviations
print(mf.averaged_product(simdict,'NONE',objsubset5,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj09sub01set5,"std_dev"))
print(mf.averaged_product(simdict,'NONE',obj08sub02set5,"std_dev"))
print(mf.averaged_product(simdict,'NONE',subobjset5,"std_dev"))

[0.17833946142861942, 0.4857768443872277]
[0.17561282362560693, 0.4831333637267715]
[0.1679189806866595, 0.4758284130657662]
[0.2140130670537146, 0.49772745953101954]


### Relation type SYNONYMS

#### Usual scalar product

In [30]:
#observable vectors
print(mf.averaged_product(simdict,'SYNONYMS',objsubset5,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set5,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set5,"mean"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset5,"mean"))

[0.14375192198193726, 0.3504889336100789]
[0.15918324261382036, 0.3530321409988211]
[0.17532173118548403, 0.34752819089127907]
[0.038156776505147004, 0.441824737290185]


In [31]:
# observable vector deviations
print(mf.averaged_product(simdict,'SYNONYMS',objsubset5,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj09sub01set5,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',obj08sub02set5,"std_dev"))
print(mf.averaged_product(simdict,'SYNONYMS',subobjset5,"std_dev"))

[0.25948428775057913, 0.4755037739873462]
[0.26825217081870095, 0.46847237202576425]
[0.27339690697044733, 0.4576974749558973]
[0.25852356391630865, 0.47955949128025155]


### Relation type ANTONYMS

#### Usual scalar product

In [32]:
#observable vectors
print(mf.averaged_product(simdict,'ANTONYMS',objsubset5,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set5,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set5,"mean"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset5,"mean"))

[0.0877816650940813, 0.34982337973197786]
[0.09136318173947341, 0.3770926606345596]
[0.11197059660329346, 0.3783293734197053]
[0.09326952951226117, 0.4530848527407711]


In [33]:
# observable vector deviations
print(mf.averaged_product(simdict,'ANTONYMS',objsubset5,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj09sub01set5,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',obj08sub02set5,"std_dev"))
print(mf.averaged_product(simdict,'ANTONYMS',subobjset5,"std_dev"))

[0.05777252540593227, 0.40597878916665514]
[0.06665069607032813, 0.4198520645713705]
[0.08584620736272351, 0.41315274358630527]
[0.12083623205043341, 0.4595268585776689]


# For final version of paper

In [86]:
vector_dic_list1 = [objsubset1,obj08sub02set1,obj05sub05set1,obj02sub08set1,subobjset1]
vector_dic_list2 = [objsubset2,obj08sub02set2,obj05sub05set2,obj02sub08set2,subobjset2]
vector_dic_list3 = [objsubset3,obj08sub02set3,obj05sub05set3,obj02sub08set3,subobjset3]
vector_dic_list4 = [objsubset4,obj08sub02set4,obj05sub05set4,obj02sub08set4,subobjset4]
vector_dic_list5 = [objsubset5,obj08sub02set5,obj05sub05set5,obj02sub08set5,subobjset5]

In [83]:
lexical = ['ANTONYMS','NONE','SYNONYMS']

In [84]:
# 13
[[mf.averaged_product(simdict,a,b,"std_dev") for a in lexical] for b in vector_dic_list1]

[[[0.05346544299522618, 0.4568929888820038],
  [0.1799884361229626, 0.5137827642037696],
  [0.26077379359520714, 0.505324436944525]],
 [[0.06697394688295825, 0.440264873207358],
  [0.16887458363159638, 0.4955873884983094],
  [0.2616326990157913, 0.4844628082587479]],
 [[0.06469236374545816, 0.464199737196295],
  [0.17394938888128803, 0.49106533923688495],
  [0.22537483515247733, 0.49690097375789877]],
 [[0.035318957623188325, 0.47335168359381496],
  [0.2046191053283457, 0.5193389422669193],
  [0.23920425070510373, 0.5111711135182679]],
 [[0.11112158739691018, 0.5345817591057888],
  [0.22536402489428037, 0.529373261707615],
  [0.27232920176431524, 0.5036461026646174]]]

In [87]:
# 23
[[mf.averaged_product(simdict,a,b,"std_dev") for a in lexical] for b in vector_dic_list2]

[[[0.08947500811193136, 0.4425619356250345],
  [0.16342154446757987, 0.5012004843230822],
  [0.2795145107377657, 0.4776719926134788]],
 [[0.08969236900972732, 0.4459821274615428],
  [0.1704649355135085, 0.4974900737294881],
  [0.2934521604205345, 0.4704812326382969]],
 [[0.12308770030944464, 0.47345751204873304],
  [0.18586960624939017, 0.5052115930789893],
  [0.30804944380070187, 0.4693431951219364]],
 [[0.12276912684075687, 0.47384958580486386],
  [0.1902614064622296, 0.515262189053194],
  [0.3118379191701441, 0.4771460186427323]],
 [[0.1383539871029549, 0.4528896305032765],
  [0.18758704579993235, 0.5150564543195832],
  [0.30580928344878666, 0.48062087590046754]]]

In [88]:
# 10
[[mf.averaged_product(simdict,a,b,"std_dev") for a in lexical] for b in vector_dic_list3]

[[[0.05346544299522618, 0.4568929888820038],
  [0.1799884361229626, 0.5137827642037696],
  [0.26077379359520714, 0.505324436944525]],
 [[0.06697394688295825, 0.440264873207358],
  [0.16887458363159638, 0.4955873884983094],
  [0.2616326990157913, 0.4844628082587479]],
 [[0.06469236374545816, 0.464199737196295],
  [0.17394938888128803, 0.49106533923688495],
  [0.22537483515247733, 0.49690097375789877]],
 [[0.035318957623188325, 0.47335168359381496],
  [0.2046191053283457, 0.5193389422669193],
  [0.23920425070510373, 0.5111711135182679]],
 [[0.11112158739691018, 0.5345817591057888],
  [0.22536402489428037, 0.529373261707615],
  [0.27232920176431524, 0.5036461026646174]]]

In [89]:
# 28
[[mf.averaged_product(simdict,a,b,"std_dev") for a in lexical] for b in vector_dic_list4]

[[[0.08416508709355439, 0.41904813625573994],
  [0.16710471272492586, 0.4897062417383025],
  [0.2751931455081116, 0.4675157793020683]],
 [[0.09236254972640569, 0.433827654561518],
  [0.17358665064083356, 0.4886746917157874],
  [0.29398182325132916, 0.46192499598847825]],
 [[0.12397821777108849, 0.46252141960990256],
  [0.1890660243278489, 0.4960721330432439],
  [0.30776770566663003, 0.46253903616283343]],
 [[0.12278084626440394, 0.4635095263763862],
  [0.1919612401510118, 0.5091702124173773],
  [0.307680110768499, 0.4742076719974423]],
 [[0.1379490720164012, 0.4466951587693989],
  [0.18928004690236228, 0.5104248827015043],
  [0.30245540076292815, 0.47888045634359644]]]

In [90]:
# 15
[[mf.averaged_product(simdict,a,b,"std_dev") for a in lexical] for b in vector_dic_list5]

[[[0.05777252540593227, 0.40597878916665514],
  [0.17833946142861942, 0.4857768443872277],
  [0.25948428775057913, 0.4755037739873462]],
 [[0.08584620736272351, 0.41315274358630527],
  [0.1679189806866595, 0.4758284130657662],
  [0.27339690697044733, 0.4576974749558973]],
 [[0.0807871534726488, 0.3948611781443709],
  [0.16931105524808, 0.45749418863072056],
  [0.2490368244140177, 0.45550435442088394]],
 [[0.05544605419754185, 0.4013679133401648],
  [0.1930797521354355, 0.4789640015347167],
  [0.24089614541873366, 0.46749789478698367]],
 [[0.12083623205043341, 0.4595268585776689],
  [0.2140130670537146, 0.49772745953101954],
  [0.25852356391630865, 0.47955949128025155]]]

## Gaussianity comparison old (only 10 theoretically predicted)

In [94]:
#object
print((objsubset3exp[0] - objsubset3.mean)/objsubset3exp[1])

[-0.20593547 -0.0531899  -0.02394437 -0.06707178 -0.05486593 -0.02046615
 -0.07910188 -0.08633712  0.06826632  0.09262384]


In [95]:
#object 09 subject 01
print((obj09sub01set3exp[0] - obj09sub01set3.mean)/obj09sub01set3exp[1])

[-0.20560064 -0.04214515 -0.04140558 -0.06292756 -0.04478231 -0.04413137
 -0.08350246 -0.06453754  0.05190338  0.0865125 ]


In [96]:
#object 08 subject 02
print((obj08sub02set3exp[0] - obj08sub02set3.mean)/obj08sub02set3exp[1])

[-0.20003557 -0.01775494 -0.0637176  -0.03205484 -0.03175714 -0.06085297
 -0.08444393 -0.02399356  0.0207836   0.05946817]


In [97]:
#subject
print((subobjset3exp[0] - subobjset3.mean)/subobjset3exp[1])

[ 0.06483219  0.10182402 -0.01819647  0.13992381  0.07306604  0.0392526
  0.0154642   0.08936283  0.04324063  0.08531824]


## Gaussianity comparison complete

In [76]:
#object
print(list(abs((objsubset5exp[0] - objsubset5.mean)/objsubset5exp[1])))

[0.20593546622032943, 0.05318990461403443, 0.02394437184217985, 0.06707177614415286, 0.05486592769188789, 0.020466148311069847, 0.0791018763795314, 0.08633712198764283, 0.06826631967511095, 0.0926238395945725, 0.1692061836687999, 0.17346085008314063, 0.1321504979194378, 0.011548821865209951, 0.06794593976012539]


In [77]:
#object 09 subject 01
print(list(abs((obj09sub01set5exp[0] - obj09sub01set5.mean)/obj09sub01set5exp[1])))

[0.20560064374401188, 0.042145154164643976, 0.041405581100799366, 0.06292756437824963, 0.04478230983513786, 0.044131371035251825, 0.08350246069864965, 0.06453754081405101, 0.051903384806587356, 0.08651249800827779, 0.1662394878368678, 0.17539632441525232, 0.14204058630168578, 0.023645865755532715, 0.032733068073192234]


In [78]:
#object 08 subject 02
print(list(abs((obj08sub02set5exp[0] - obj08sub02set5.mean)/obj08sub02set5exp[1])))

[0.20003556850367124, 0.01775494069784047, 0.06371759526367382, 0.032054835811967115, 0.031757141751285245, 0.06085296751092967, 0.08444392601273537, 0.02399356498797485, 0.020783604731099584, 0.0594681707724693, 0.16633984418953446, 0.18146408203140327, 0.16608573283739717, 0.04822598354563592, 0.021534385584652507]


In [79]:
#subject
print(list(abs((subobjset5exp[0] - subobjset5.mean)/subobjset5exp[1])))

[0.06483219180589411, 0.10182402231721392, 0.018196473983125197, 0.13992380945380137, 0.07306604248920741, 0.03925260498590186, 0.015464202621115055, 0.08936283244574272, 0.043240632775038194, 0.08531824090716984, 0.13089087075562927, 0.1549432518589387, 0.0481611038370049, 0.0518095361203826, 0.04091442354923745]


In [80]:
#object 05 subject 05
print(list(abs((obj05sub05set5exp[0] - obj05sub05set5.mean)/obj05sub05set5exp[1])))

[0.05724504003655622, 0.07830475476292152, 0.08750253785506035, 0.11820612104170361, 0.02601775659593383, 0.013247212059080766, 0.06015788849961322, 0.04875697291936739, 0.015853264286211335, 0.04813172609366606, 0.159254232122931, 0.1699618365170699, 0.16173457556705748, 0.06172513376727118, 0.04993012785192271]


In [81]:
#object 02 subject 08
print(list(abs((obj02sub08set5exp[0] - obj02sub08set5.mean)/obj02sub08set5exp[1])))

[0.049722295621177914, 0.09681974169152384, 0.07054755733908176, 0.13766082630197404, 0.07023618383288582, 0.03924677443556081, 0.007487335346656932, 0.08223682566186251, 0.02619918563949922, 0.07770134338322122, 0.13529304455403623, 0.15586713260853605, 0.05659418888996559, 0.06055823814882146, 0.05527430452618728]


In [98]:
print(x4)

[ 1.49126287e-02 -3.35713396e-03  3.03210380e-03 -1.28909688e-04
  1.73749047e-03  2.38531350e-03  3.80771074e-05  2.52787197e-06
  2.13516410e-03 -3.16706705e-05  2.19091199e-03  2.12097697e-03
  2.13797682e-03]
